In [114]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [115]:
# parameters of example 3.5
grid_width = 5
grid_height = 5

actions= np.array([[-1,0], [1, 0], [0, -1], [0, 1]])
state_A = np.array([1, 0])
state_B = np.array([3, 0])
state_A_prime = np.array([1, grid_height-1])
state_B_prime = np.array([3, 2])

reward_offset = 0
reward_any = 0 + reward_offset
reward_out = -1 + reward_offset
reward_A = 10 + reward_offset
reward_B = 5 + reward_offset
gamma = 0.9
prob_policy = 1. / len(actions)

theta = 1e-3

In [116]:
# figure 3.5
num_steps = 100
values_prime = np.zeros([grid_height, grid_width])
values = np.zeros([grid_height, grid_width])
for step in range(num_steps):
    for y in range(grid_height):
        for x in range(grid_width):
            state = np.array([x, y])
            values_actions = np.zeros(actions.shape[0])
            for a, action in enumerate(actions):
                state_prime = state + action
                reward = reward_any
                if np.all(state == state_A):
                    state_prime = state_A_prime
                    reward = reward_A
                elif np.all(state == state_B):
                    state_prime = state_B_prime
                    reward = reward_B
                elif np.any(state_prime < np.array([0, 0])) or np.any(state_prime >= np.array([grid_width, grid_height])):
                    state_prime = state
                    reward = reward_out
                values_actions[a] = reward + gamma * values[state_prime[1], state_prime[0]]
            values_prime[y, x] = np.max(values_actions)
    delta = np.linalg.norm(values_prime - values)
    values = np.copy(values_prime)
    if delta < theta:
        print('converged after {} timesteps'.format(step))
        break
    
print(values)

converged after 96 timesteps
[[21.97690153 24.41877948 21.97690153 19.41877948 17.47690153]
 [19.77884703 21.97690153 19.77884703 17.80096232 16.02086609]
 [17.80096232 19.77884703 17.80096232 16.02086609 14.41877948]
 [16.02086609 17.80096232 16.02086609 14.41877948 12.97690153]
 [14.41877948 16.02086609 14.41877948 12.97690153 11.67884703]]


In [117]:
# example 3.5
def state_value_function(state, value):
    v_new = 0
    for action in actions:
        state_prime = state + action
        reward = reward_any
        if np.all(state == state_A):
            state_prime = state_A_prime
            reward = reward_A
        elif np.all(state == state_B):
            state_prime = state_B_prime
            reward = reward_B
        elif np.any(state_prime < np.array([0, 0])) or np.any(state_prime >= np.array([grid_width, grid_height])):
            state_prime = state
            reward = reward_out
        v_new += prob_policy * (reward + gamma * values[state_prime[1], state_prime[0]])
    return v_new

num_steps = 100
values = np.zeros([grid_height, grid_width])
for step in range(num_steps):
    values_prime = np.zeros([grid_height, grid_width])
    for y in range(grid_height):
        for x in range(grid_width):
            state = np.array([x, y])
            values_prime[y, x] = state_value_function(state, values[y, x])
    delta = np.linalg.norm(values_prime - values)
    if delta < theta:
        print('converged after {} timesteps'.format(step))
        break
    values = values_prime
    
print(values)
values_35 = values


converged after 39 timesteps
[[ 3.31090387  8.79118019  4.42945778  5.32416158  1.49394036]
 [ 1.52349102  2.99419728  2.25197626  1.90936483  0.54916766]
 [ 0.05272354  0.74004854  0.67495194  0.35998538 -0.40136713]
 [-0.97168967 -0.43361477 -0.35303741 -0.58379676 -1.18128939]
 [-1.85579583 -1.34334742 -1.22741763 -1.42110288 -1.97338525]]


In [118]:
# example 3.5 with in-place update
def state_value_function(state, value):
    v_new = 0
    for action in actions:
        state_prime = state + action
        reward = reward_any
        if np.all(state == state_A):
            state_prime = state_A_prime
            reward = reward_A
        elif np.all(state == state_B):
            state_prime = state_B_prime
            reward = reward_B
        elif np.any(state_prime < np.array([0, 0])) or np.any(state_prime >= np.array([grid_width, grid_height])):
            state_prime = state
            reward = reward_out
        v_new += prob_policy * (reward + gamma * values[state_prime[1], state_prime[0]])
    return v_new   

num_steps = 100
values = np.zeros([grid_height, grid_width])
for step in range(num_steps):
    delta = 0.0
    for y in range(grid_height):
        for x in range(grid_width):
            state = np.array([x, y])
            v_current = values[y, x]
            v_new = state_value_function(state, v_current)
            values[y, x] = v_new
            delta = max(delta, np.linalg.norm(v_new - v_current))
    if delta < theta:
        print('converged after {} timesteps'.format(step))
        break
    
print(values)
values_35_ip = values

converged after 29 timesteps
[[ 3.31359559  8.79292942  4.43113177  5.32556099  1.4955287 ]
 [ 1.52582318  2.99591435  2.2534199   1.91064941  0.55045095]
 [ 0.05486787  0.74165922  0.67626363  0.36114423 -0.40025498]
 [-0.96965064 -0.43208514 -0.35180898 -0.58272448 -1.18027658]
 [-1.85380443 -1.34185832 -1.22622928 -1.42007309 -1.97241846]]


In [119]:
# compare values from 3.5 with 3.5 with in-place update
delta = np.linalg.norm(values_35 - values_35_ip)
print(delta)

0.00791342367110317


In [141]:
# example 4.1
num_steps = 20
r_any = -1
terminal_states = np.array([[0, 0], [grid_width - 1, grid_height - 1]])

in_place = False
print_values_after_every_step = True

def state_value_function(state, value):
    v_new = 0
    for action in actions:
        state_prime = state + action
        reward = r_any
        if np.any(state_prime < np.array([0, 0])) or np.any(state_prime >= np.array([grid_width, grid_height])):
            state_prime = state
        v_new += prob_policy * (reward + gamma * values[state_prime[1], state_prime[0]])
    return v_new   

values = np.zeros([grid_height, grid_width])
for step in range(num_steps):
    delta = 0.0
    values_prime = np.zeros([grid_height, grid_width]) if not in_place else values
    for y in range(grid_height):
        for x in range(grid_width):
            state = np.array([x, y])
            if np.any(np.all(terminal_states == state, axis = 1)):
                continue
            v_current = values[y, x]
            v_new = state_value_function(state, v_current)
            values_prime[y, x] = v_new
            delta = max(delta, np.linalg.norm(v_new - v_current))
    values = np.copy(values_prime)
    if print_values_after_every_step:
        print('step ', step)
        print(values)
    if delta < theta:
        print('converged after {} timesteps'.format(step))
        break
print('final values')
print(values, prob_policy)


step  0
[[ 0. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1.  0.]]
step  1
[[ 0.    -1.675 -1.9   -1.9   -1.9  ]
 [-1.675 -1.9   -1.9   -1.9   -1.9  ]
 [-1.9   -1.9   -1.9   -1.9   -1.9  ]
 [-1.9   -1.9   -1.9   -1.9   -1.675]
 [-1.9   -1.9   -1.9   -1.675  0.   ]]
step  2
[[ 0.       -2.231875 -2.659375 -2.71     -2.71    ]
 [-2.231875 -2.60875  -2.71     -2.71     -2.71    ]
 [-2.659375 -2.71     -2.71     -2.71     -2.659375]
 [-2.71     -2.71     -2.71     -2.60875  -2.231875]
 [-2.71     -2.71     -2.659375 -2.231875  0.      ]]
step  3
[[ 0.         -2.6875     -3.32003125 -3.42760938 -3.439     ]
 [-2.6875     -3.22384375 -3.40482812 -3.439      -3.42760938]
 [-3.32003125 -3.40482812 -3.439      -3.40482813 -3.32003125]
 [-3.42760938 -3.439      -3.40482812 -3.22384375 -2.6875    ]
 [-3.439      -3.42760938 -3.32003125 -2.6875      0.        ]]
step  4
[[ 0.         -3.07705937 -3.88899297 -4.06576914 -4.08997422]
 [-3.077